Analysis pinned to Kernel version 5.10

Outdated, no longer using ctags/cscope

In [1]:
#todo: make the regexes more strict.

In [2]:
import re
import itertools
from itertools import islice
from subprocess import run

from pathlib import Path
import sqlite3

from locations import *


connection = sqlite3.connect(OUTDIR/"function_survey.db")
cursor = connection.cursor()

In [3]:
def parse(filename, expression):
    with open(filename) as f:
        return [re.match(expression, line) for line in f]
    
def head(iterable, n=10):
    return list(islice(iterable, n))

## Ctags

In [4]:
def reset_ctags_db():
    cursor.execute("CREATE TABLE IF NOT EXISTS ctags (file, name, line, token_type, text)")
    cursor.execute("DELETE FROM ctags")
    
def parse_ctags(filename):
    expression = r"(?P<name>^[^ ]+)\s+(?P<token_type>[^ ]+)\s+(?P<linenum>\d+) (?P<path>[^ ]+) (?P<text>.*)"
    return parse(filename, expression)

In [6]:
list(cursor.execute("SELECT * FROM ctags LIMIT 10"))

[]

In [ ]:
# List all c and header files
!find -name *.c -o -name *.h > {all_c_code}
# get all tokens in the kernel.
!ctags -x -L {all_c_code} > {kernel_tags}

In [ ]:
# sort ctags output by line number, give contents of each function. (C does not allow nested functions)
# assume one function per line.

#the --languages=C flag causes headers to be excluded by ctags
#running ctags with no settings results in python files being included

In [12]:
reset_ctags_db()
cursor.executemany("INSERT INTO ctags VALUES (?,?,?,?,?)", (
    (m["path"][len("./"):],
     m["name"], m["linenum"], m["token_type"], m["text"])
    for m in parse_ctags(kernel_tags)))

In [13]:
head(cursor.execute("SELECT * FROM ctags WHERE token_type = 'function'"))

[('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_A220_VSC_BIN_SIZE_HEIGHT',
  '1419',
  'function',
  'static inline uint32_t A2XX_A220_VSC_BIN_SIZE_HEIGHT(uint32_t val)'),
 ('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_A220_VSC_BIN_SIZE_WIDTH',
  '1413',
  'function',
  'static inline uint32_t A2XX_A220_VSC_BIN_SIZE_WIDTH(uint32_t val)'),
 ('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_CLEAR_COLOR_ALPHA',
  '2397',
  'function',
  'static inline uint32_t A2XX_CLEAR_COLOR_ALPHA(uint32_t val)'),
 ('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_CLEAR_COLOR_BLUE',
  '2391',
  'function',
  'static inline uint32_t A2XX_CLEAR_COLOR_BLUE(uint32_t val)'),
 ('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_CLEAR_COLOR_GREEN',
  '2385',
  'function',
  'static inline uint32_t A2XX_CLEAR_COLOR_GREEN(uint32_t val)'),
 ('drivers/gpu/drm/msm/adreno/a2xx.xml.h',
  'A2XX_CLEAR_COLOR_RED',
  '2379',
  'function',
  'static inline uint32_t A2XX_CLEAR_COLOR_RED(uint32_t val)'),
 ('drivers/gpu

In [ ]:
# files = {}
# test_functions = set()
# for m in parse_ctags(kernel_tags):
#     # name token_type linenum path text
#     files[m["path"]] = files.get(m["path"],[]) + [m]
#     if m["token_type"] == "function" and "test" in m["name"]:
#         test_functions.add(m)

In [ ]:
# {k: 
#  [(int(m["linenum"]), m["token_type"],
#    m["name"], m["text"])
#   for m in sorted(v, key=lambda m:int(m["linenum"]))]
#  for k,v in islice(files.items(), 1, 2)}

In [ ]:
# [(int(m["linenum"]), m["token_type"],
#    m["name"], m["text"])
#   for m in sorted(files["./fs/9p/fid.c"],
#                   key=lambda m:int(m["linenum"]))]

In [ ]:
# !grep -v variable {kernel_tags} | head -n 50 

## Cscope

In [ ]:
def parse_cscope(filename):
    expression = r"(?P<path>^[^ ]+) (?P<funcname>[^ ]+) (?P<linenum>\d+) (?P<usage_line>.*)"
    return parse(filename, expression)

In [ ]:
# get all function calls
!cscope -RL2 ".*" > {all_calls}
# get all calls from a function with test in the name
!cscope -RL2 ".*test.*" > {test_targets}

In [ ]:
called_functions = set()
called_function_names = set()
for m in parse_cscope(all_calls):
    called_functions.add((m["funcname"], m["path"]))
    called_function_names.add(m["funcname"])

In [ ]:
tested_functions = set()
tested_function_names = set()     
for m in parse_cscope(test_targets):
    tested_functions.add((m["funcname"], m["path"]))
    tested_function_names.add(m["funcname"])

In [ ]:
len(tested_function_names - called_function_names), tested_function_names - called_function_names
# before the math.c fix, there were 187 functions in this set

In [ ]:
# functions detected by cscope (getting called by the tests) but not by ctags
len(tested_function_names - func_names), len(tested_functions - functions)

In [ ]:
target = KERNEL

!(cd {target}; cscope -RL0 ".*")

In [ ]:
# the results of this cell seem to indicate that a single directory (arch/sh/math-emu) is tripping up cscope.
# removeing that file lets cscope run unimpeded,
def test_cscope(path):
    if not path.is_dir():
        print("fail on file:",path)
        return
    for p in path.iterdir():
        if p.is_dir():
            print(p)
            result = run('cscope -RL0 ".*"',cwd=p, capture_output=True, shell=True)
            # return code is a better way to identify if errors
            # could just list all dirs/files in kernel, and echo stderr to each
            # find -d (gives list of directories)
            if result.stderr not in {b'', b'cscope: no source files found\n'}:
                print(p, repr(result.stderr))
                test_cscope(p)

test_cscope(KERNEL)

In [ ]:
result = run('cscope -RL0 ".*"', capture_output=True, shell=True)
result